In [ ]:
import numpy as np
import pandas as pd
import math
import copy
from config import *

class SystemModel:
    def __init__(self):
        self.UAVspeed = UAV_Speed
        self.User_number = NumberOfUsers
        self.UserperCell = UserNumberPerCell
        
        # Initialize UAV positions
        self.PositionOfUAVs = pd.DataFrame(
            np.zeros((3, NumberOfUAVs)),
            index=['x', 'y', 'z'],
            columns=[f'UAV{i}' for i in range(NumberOfUAVs)]
        )
        self._init_uav_positions()
        
        # Initialize user positions
        self.PositionOfUsers = pd.DataFrame(
            np.zeros((3, NumberOfUsers)),
            index=['x', 'y', 'z'],
            columns=[f'U{i}' for i in range(NumberOfUsers)]
        )
        self._init_user_positions()
        
        # Service zone borders
        self.Zone_border_X = ServiceZone_X
        self.Zone_border_Y = ServiceZone_Y
        self.Zone_border_Z = Height_limit_Z
        
        # Store initial positions and state
        self.Init_PositionOfUsers = copy.deepcopy(self.PositionOfUsers)
        self.Init_PositionOfUAVs = copy.deepcopy(self.PositionOfUAVs)
        self.State = np.zeros([1, NumberOfUAVs * 3 + NumberOfUsers], dtype=float)
        
        # Power allocation
        self.Power_allocation_list = pd.DataFrame(
            np.ones((1, NumberOfUsers)) * UAV_power_unit,
            columns=np.arange(NumberOfUsers).tolist()
        )

    def _init_uav_positions(self):
        self.PositionOfUAVs.iloc[0,:] = [225, 400]  # x coordinates
        self.PositionOfUAVs.iloc[1,:] = [250, 500]  # y coordinates
        self.PositionOfUAVs.iloc[2,:] = [100, 100]  # z coordinates

    def _init_user_positions(self):
        self.PositionOfUsers.iloc[0,:] = [250, 200, 375, 383]  # x coordinates
        self.PositionOfUsers.iloc[1,:] = [200, 300, 375, 426]  # y coordinates
        self.PositionOfUsers.iloc[2,:] = 0  # z coordinates (ground level)

    def user_movt(self):
        """Update user movement pattern"""
        self.PositionOfUsers.iloc[[0,1], 2] -= 0.75
        return self.PositionOfUsers

    def Get_Distance_U2K(self, UAV_Position, User_Position, UAVsnumber, Usersnumber):
        """Calculate distances between UAVs and users"""
        Distance = pd.DataFrame(
            np.zeros((UAVsnumber, Usersnumber)),
            columns=np.arange(Usersnumber).tolist()
        )
        
        for i in range(UAVsnumber):
            for j in range(Usersnumber):
                Distance.iloc[i,j] = np.linalg.norm(UAV_Position.iloc[:,i]-User_Position.iloc[:,j])
        
        return Distance

    def Get_Propagation_Loss(self, Distance_U2K, UAV_Position, UAVsnumber, Usersnumber, f_c):
        """Calculate propagation loss with LoS/NLoS considerations"""
        Propagation_Loss = pd.DataFrame(
            np.zeros((UAVsnumber, Usersnumber)),
            columns=np.arange(Usersnumber).tolist()
        )

        for i in range(UAVsnumber):
            for j in range(Usersnumber):
                UAV_Height = UAV_Position.iloc[2,i]
                D_H = np.sqrt(np.square(Distance_U2K.iloc[i,j])-np.square(UAV_Height))
                d_0 = np.max([(294.05*math.log(UAV_Height,10)-432.94),18])
                p_1 = 233.98*math.log(UAV_Height,10) - 0.95
                
                # Calculate LoS probability
                if D_H <= d_0:
                    P_Los = 1.0
                else:
                    P_Los = d_0/D_H + math.exp(-(D_H/p_1)*(1-(d_0/D_H)))
                P_Los = min(P_Los, 1)
                P_NLos = 1 - P_Los
                
                # Calculate losses
                L_Los = 30.9 + (22.25-0.5*math.log(UAV_Height,10))*math.log(Distance_U2K.iloc[i,j],10) + 20*math.log(f_c,10)
                L_NLos = np.max([L_Los,32.4+(43.2-7.6*math.log(UAV_Height,10))*math.log(Distance_U2K.iloc[i,j],10)+20*math.log(f_c,10)])
                Avg_Los = P_Los*L_Los + P_NLos*L_NLos
                
                # Add Rayleigh fading
                gain = np.random.rayleigh(scale=1)*pow(10,(-Avg_Los/10))
                Propagation_Loss.iloc[i,j] = gain
                
        return Propagation_Loss

    # Additional methods (SINR calculation, data rate calculation, etc.) follow the same pattern...